# Preparation for the moodle exercise in Spark

In this jupyter notebook we are going to make the preprocessing part of the dataset that is going to be used in the graded exercise of this week.
1. Change to exercise08 repository

2. Start docker <br>
docker-compose up -d

3. Getting the data:
Follow the procedure that is described below. The dataset can be found here: http://data.greatlanguagegame.com.s3.amazonaws.com/confusion-2014-03-02.tbz2. More specifically do the following:
- download the data      :<br> ```wget http://data.greatlanguagegame.com.s3.amazonaws.com/confusion-2014-03-02.tbz2```
- extract the data       :<br> ```tar -jxvf confusion-2014-03-02.tbz2```

4. copy the data to hdfs :<br> ```docker cp confusion-2014-03-02/confusion-2014-03-02.json jupyter:/home/jovyan/work``` <br>
(Copying the data to hdfs needs to be done only once and it might take 1-2 minutes.)

For more information about the dataset, you can refer to https://lars.yencken.org/datasets/great-language-game/

## Preprocessing commands
In your newly created notebook run these commands in order to have the dataset into an RDD:

In [1]:
import json
from pyspark.sql import SparkSession
from pyspark import SparkConf

spark = SparkSession.builder.master('local').getOrCreate()
sc = spark.sparkContext

path = "confusion-2014-03-02.json"
raw_data = sc.textFile(path)
dataset = raw_data.map(json.loads).cache()

After that you will be able to run the queries of the moodle question of this week. The RDD that you have to perform your queries on is the ```dataset``` one. For example, the following command returns one element of the dataset:

In [2]:
dataset.take(1)

[{'guess': 'Norwegian',
  'target': 'Norwegian',
  'country': 'AU',
  'choices': ['Maori', 'Mandarin', 'Norwegian', 'Tongan'],
  'sample': '48f9c924e0d98c959d8a6f1862b3ce9a',
  'date': '2013-08-19'}]

In [32]:
quest1 = dataset.filter(lambda obj : obj["guess"] == obj["target"] and obj["target"] == "Russian").cache()
quest1.count()

290818

In [33]:
quest2 = dataset.map(lambda obj : obj["target"] ).cache()
quest2dist = quest2.distinct()
quest2dist.count()

78

In [7]:

quest3sorted = quest1.sortBy( lambda obj : (obj["target"], obj["country"], obj["date"]))  
quest3sorted.take(10)

[{'guess': 'Albanian',
  'target': 'Albanian',
  'country': 'A1',
  'choices': ['Albanian', 'Macedonian'],
  'sample': '00b85faa8b878a14f8781be334deb137',
  'date': '2013-09-04'},
 {'guess': 'Albanian',
  'target': 'Albanian',
  'country': 'A1',
  'choices': ['Albanian', 'Bulgarian', 'Indonesian', 'Portuguese'],
  'sample': 'efcd813daec1c836d9f030b30caa07ce',
  'date': '2013-09-05'},
 {'guess': 'Albanian',
  'target': 'Albanian',
  'country': 'A1',
  'choices': ['Albanian', 'Hindi', 'Swahili'],
  'sample': '13722ceed1eede7ba597ade9b4cb9807',
  'date': '2013-09-08'},
 {'guess': 'Albanian',
  'target': 'Albanian',
  'country': 'A1',
  'choices': ['Albanian', 'Tamil'],
  'sample': 'efcd813daec1c836d9f030b30caa07ce',
  'date': '2013-09-08'},
 {'guess': 'Albanian',
  'target': 'Albanian',
  'country': 'A1',
  'choices': ['Albanian', 'Urdu'],
  'sample': 'efcd813daec1c836d9f030b30caa07ce',
  'date': '2013-09-08'},
 {'guess': 'Albanian',
  'target': 'Albanian',
  'country': 'A1',
  'choices':

In [12]:
quest4 = dataset.map(lambda obj : ( (obj["country"], obj["target"] ), 1) )
quest4red = quest4.reduceByKey(lambda x, y: x+y)
quest4sorted = quest4red.sortBy( keyfunc = lambda obj : obj[1], ascending = False)  
quest4sorted.take(10)

[(('US', 'French'), 112934),
 (('US', 'German'), 112007),
 (('US', 'Spanish'), 110919),
 (('US', 'Italian'), 109603),
 (('US', 'Russian'), 107789),
 (('US', 'Japanese'), 106689),
 (('US', 'Korean'), 106442),
 (('US', 'Mandarin'), 105427),
 (('US', 'Vietnamese'), 103545),
 (('US', 'Cantonese'), 102357)]

In [13]:
quest5 = quest1.filter(lambda obj : obj["guess"] == obj["choices"][0])
quest5count = quest5.count()
datasetcount = dataset.count()

print("quest5count: ",quest5count)
print("datasetcount: ",datasetcount)
print("quest5count / datasetcount: ",quest5count / datasetcount)


quest5count:  4227531
datasetcount:  16511224
quest5count / datasetcount:  0.25603983084476356


In [27]:
quest6 = quest1.map(lambda obj : (obj["target"], 1))
quest6red = quest6.reduceByKey(lambda x, y: x+y)
quest6sorted = quest6red.sortBy( keyfunc = lambda obj : obj[1], ascending = False)  
sol1 = quest6sorted.take(10)
print(sol1)

#(language, (counter, correctCounter))
quest6Perc = dataset.map(lambda obj : (obj["target"], (1, 1 if obj["target"] == obj["guess"] else 0 ) ) )
quest6redPre = quest6Perc.reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1]) )
quest6redPerc = quest6redPre.mapValues(lambda obj : obj[1][1]/obj[1][0] )
quest6sortedPerc = quest6redPerc.sortBy( keyfunc = lambda obj : obj[1], ascending = False)  
sol2 = quest6sortedPerc.take(10)
print(sol2) #lol non funge e non capisco l'errore, ho messo la risposta con il primo print
#not sure what it means by overall?, 
#if i english was the right answer in 100 games, and I have chosen it correctly 50 -> 50%
#while italian was the right answer in 10 games and I have chosen it correctly 10 times -> 100%

#Should i choose English (50>10) or Italian (100%>50%)

#this solution is if  the 

[('French', 325430), ('German', 315271), ('Spanish', 304147), ('Italian', 300881), ('Russian', 290818), ('Korean', 283090), ('Japanese', 282007), ('Mandarin', 278381), ('Vietnamese', 267440), ('Cantonese', 262619)]


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 231.0 failed 1 times, most recent failure: Lost task 0.0 in stage 231.0 (TID 7744) (00af957a9afd executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 594, in process
    out_iter = func(split_index, iterator)
  File "/usr/local/spark/python/pyspark/rdd.py", line 2916, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/spark/python/pyspark/rdd.py", line 2916, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/spark/python/pyspark/rdd.py", line 2916, in pipeline_func
    return func(split, prev_func(split, iterator))


In [30]:
datasetDateSorted = dataset.sortBy( keyfunc = lambda obj : obj["date"], ascending = False)
lastdateObj = datasetDateSorted.take(1)
print("lastdateObj : ", lastdateObj)
lastdate = lastdateObj[0]["date"]
print("lastdate : ", lastdate)
quest7 = datasetDateSorted.filter( lambda obj : obj["date"] == lastdate)
quest7.count()

lastdateObj :  [{'guess': 'Swedish', 'target': 'Swedish', 'country': 'DE', 'choices': ['Danish', 'Swedish'], 'sample': '6640b78667f0ab23077e4a0b51568f89', 'date': '2014-03-01'}]
lastdate :  2014-03-01


65653